<a href="https://colab.research.google.com/github/Caloca24/Pycaret_Udemy_Course/blob/main/CustomerLifetimeValue_CustomerSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install pycaret
!pip install pycaret -q

In [2]:
from pycaret.datasets import get_data
df=get_data('germany')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662,Germany
1,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662,Germany
2,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662,Germany
3,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662,Germany
4,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662,Germany


In [18]:
df.dtypes

,0
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
UnitPrice,float64
CustomerID,int64
Country,object
customer_monetary_value,float64
TotalAmount,float64


In [17]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Calculate Customer Lifetime Value based on Customer Segmentation

In [12]:
import pandas as pd
import numpy as np

# Calculate the total monetary value for each customer
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']
customer_monetary_value = df.groupby('customer_monetary_value').sum()

customer_monetary_value = customer_monetary_value[['CustomerID' , 'Quantity', 'UnitPrice', 'TotalAmount']]


In [13]:
# Define segments based on total monetary value
segments = pd.qcut(customer_monetary_value['TotalAmount'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

# Add the segments to the customer_monetary_value DataFrame
customer_monetary_value['Segment'] = segments


In [14]:
customer_monetary_value.head()

,CustomerID,Quantity,UnitPrice,TotalAmount,Segment
customer_monetary_value,,,,,
-599.50,13810,-1,599.50,-599.50,Low
-549.34,12473,-1,549.34,-549.34,Low
-424.06,12473,-1,424.06,-424.06,Low
-389.68,12619,-1,389.68,-389.68,Low
-229.95,12471,-21,10.95,-229.95,Low


In [15]:
b# Calculate the average CLV for each segment
clv_by_segment = customer_monetary_value.groupby('Segment')['TotalAmount'].mean()

# Print the CLV by segment
print(clv_by_segment)

Segment
Low           -43.408606
Medium         26.363293
High          150.368780
Very High    1211.373152
Name: TotalAmount, dtype: float64


In [19]:
#filtering data
rfm_data = df[['CustomerID', 'InvoiceDate', 'TotalAmount']]

#calculate FRM Metrics
recency = (df['InvoiceDate'].max() - rfm_data.groupby('CustomerID')['InvoiceDate'].max()).dt.days
frequency = rfm_data.groupby('CustomerID')['InvoiceDate'].nunique()
moneytary_value = rfm_data.groupby('CustomerID')['TotalAmount'].sum()

#assign scores
rfm_scores = pd.DataFrame({'Recency':recency, 'Frequency':frequency, 'MonetaryValue' : moneytary_value, 'InvoiceDate': df.groupby('CustomerID')['InvoiceDate'].max()})

In [21]:
#spliting
from datetime import timedelta
train_end_date = df['InvoiceDate'].max() - timedelta(days=90)
train_data = rfm_scores[rfm_scores['InvoiceDate'] <= train_end_date]
train_data = rfm_scores[rfm_scores['InvoiceDate'] > train_end_date]


In [23]:
#prepare for training
X_train = train_data[['Recency', 'Frequency', 'MonetaryValue']]
y_train = train_data['MonetaryValue']

In [24]:
#model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [25]:
#extract predicition date
prediction_date = df['InvoiceDate'].max() + timedelta(days = 90)

In [27]:
#prepare for prediction
prediction_data = rfm_scores[rfm_scores['InvoiceDate'] <= prediction_date]

#Prepare features
X_pred = prediction_data[['Recency', 'Frequency', 'MonetaryValue']]

#predicition
clv_predicitions = model.predict(X_pred)